# Get the dataset


In [ ]:
import csv
import numpy as np
import config

test_image = np.load(config.t_path)
test_names = np.load(config.n_path)

assert(test_image.shape[0] == test_names.shape[0])


# Boilerplate Code

In [ ]:
import torch
from typing import Tuple
class Data(torch.utils.data.Dataset):
    def __init__(self, image: np.ndarray, name: np.ndarray) -> None:
        assert(image.shape[0] == name.shape[0])
        self.image = image
        self.name = name
    
    def __len__(self) -> int:
        return self.image.shape[0]
    
    def __getitem__(self, idx: int) -> 'Tuple[np.ndarray, str]':
        return self.image[idx], self.name[idx]


# Model

In [ ]:
model = config.get_model()
model.load_state_dict(torch.load(config.model_path))

use_cuda = torch.cuda.is_available()
if not use_cuda:
    print("CUDA not used!")
device = torch.device("cuda" if use_cuda else "cpu")

model = model.to(device)


# Run tests

In [ ]:
from tqdm import tqdm

tests = Data(test_image, test_names)
tests_dataloader = torch.utils.data.DataLoader(
    tests, batch_size = config.batch_size)


with torch.no_grad(), open(config.result_path, "w", newline='') as result:
    result_writer = csv.writer(result)
    for image, names in tqdm(tests_dataloader):
        image = image.to(device, dtype=torch.float)

        output = model(image)
        output = output.argmax(dim=1).cpu().item()
        assert(output.shape[0] == names.shape[0])
        
        result_writer.writerows(zip(names, output))


# Submit


In [5]:
from datetime import datetime
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

choice = input(f"Data saved at {config.result_path}. Submit?")
if choice == "":
    api.competition_submit(config.result_path, f"Submitted with hope and joy at {datetime.now()}", "hsgs-hackathon2022")

/home/phuonghd/.conda/envs/nhatenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HEHE
